# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [72]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
import pickle
import nltk
import re
from sklearn.model_selection import train_test_split
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [83]:
# load data from database
engine = create_engine('sqlite:///etl_pipeline.db')
print(engine.table_names())
df = pd.read_sql ('SELECT * FROM messageCat', engine)
df.head(20)
#df2= pd.read_sql ('SELECT * FROM messageCat', engine)
#df2.head()

['messageCat', 'messageCat1', 'messageCat2', 'messageCat3', 'messageCatlight']


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2.0,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7.0,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8.0,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9.0,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12.0,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,14.0,Information about the National Palace-,Informtion au nivaux palais nationl,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,15.0,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,16.0,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
8,17.0,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,18.0,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1


In [75]:
X = df['message']
print(X)
y = df.iloc[:, 4:39]
#print(Y)
y.columns.tolist()
#print(y)

0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
5                   Information about the National Palace-
6                           Storm at sacred heart of jesus
7        Please, we need tents and water. We are in Sil...
8          I would like to receive the messages, thank you
9        I am in Croix-des-Bouquets. We have health iss...
10       There's nothing to eat and water, we starving ...
11       I am in Petionville. I need more information r...
12       I am in Thomassin number 32, in the area named...
13       Let's do it together, need food in Delma 75, i...
14       More information on the 4636 number in order f...
15       A Comitee in Delmas 19, Rue ( street ) Janvier...
16       We need food and water in Klecin 12. We are dy.

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather']

In [ ]:
### 2. Write a tokenization function to process your text data

In [76]:
#def tokenize(text):
def tokenize(texts):
# Remove non-alphanumeric characters
    texts = re.sub(r'[^\w\s]', '', texts)
    
    # Tokenize the text
    tokens = word_tokenize(texts)
    
    # Initialize the lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # Lemmatize and clean each token
    clean_tokens = [lemmatizer.lemmatize(tok).strip() for tok in tokens]
    
    return clean_tokens

#print(X)
#print(tokenize(x))
# Example usage
for message in X[:56]:
    print(message)
    print(tokenize(message),'\n')
    

Weather update - a cold front from Cuba that could pass over Haiti
['Weather', 'update', 'a', 'cold', 'front', 'from', 'Cuba', 'that', 'could', 'pas', 'over', 'Haiti'] 

Is the Hurricane over or is it not over
['Is', 'the', 'Hurricane', 'over', 'or', 'is', 'it', 'not', 'over'] 

Looking for someone but no name
['Looking', 'for', 'someone', 'but', 'no', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['UN', 'report', 'Leogane', '8090', 'destroyed', 'Only', 'Hospital', 'St', 'Croix', 'functioning', 'Needs', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'of', 'Haiti', 'rest', 'of', 'the', 'country', 'today', 'and', 'tonight'] 

Information about the National Palace-
['Information', 'about', 'the', 'National', 'Palace'] 

Storm at sacred heart of jesus
['Storm', 'at', 'sacred', 'heart', 'of', 'jesus'] 

Please, we need tents and water. We are in Silo, Thank yo

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [77]:

pipeline = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=tokenize)),  
   ('tfidf_transformer', TfidfTransformer()),           
    ('classifier', MultiOutputClassifier(RandomForestClassifier()))  
])

# Replace None or NaN in X with an empty string
X = X.fillna(" ")

print(X)
print("Are there missing values in y?", pd.isnull(y).any())

0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
5                   Information about the National Palace-
6                           Storm at sacred heart of jesus
7        Please, we need tents and water. We are in Sil...
8          I would like to receive the messages, thank you
9        I am in Croix-des-Bouquets. We have health iss...
10       There's nothing to eat and water, we starving ...
11       I am in Petionville. I need more information r...
12       I am in Thomassin number 32, in the area named...
13       Let's do it together, need food in Delma 75, i...
14       More information on the 4636 number in order f...
15       A Comitee in Delmas 19, Rue ( street ) Janvier...
16       We need food and water in Klecin 12. We are dy.

In [78]:
### 4. Train pipeline
#- Split data into train and test sets
#- Train pipeline
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=3)
# Fit the pipeline to the training data
pipeline.fit(X_train, y_train)
print(X_train.head())
print(X_test.head())
print(y_train)
y.sum() /len(y)


6488            I need a shelter, what can you do for me? 
154      good evening we ? carrefour feuill ???we are d...
16205    Part of the money will be used to work with ot...
12431    Auntie Em Auntie Em!!!! (@Frankenstorm Apocaly...
25042    Apparently oblivious to the tension, young Som...
Name: message, dtype: object
21258    Thousands are stranded in their homes without ...
23394    Usually July is the peak month for vivax malar...
19086    Many maritime delimitations, in particular of ...
15024    It is this contaminated water that has raised ...
10688            How do #hurricanes get their names ? .. .
Name: message, dtype: object
       related  request  offer  aid_related  medical_help  medical_products  \
6488         1        0      0            0             0                 0   
154          1        1      0            1             0                 0   
16205        1        1      0            1             0                 0   
12431        1        0      0      

related                   0.774002
request                   0.170680
offer                     0.004534
aid_related               0.414432
medical_help              0.079511
medical_products          0.050061
search_and_rescue         0.027583
security                  0.017944
military                  0.032764
child_alone               0.000000
water                     0.063776
food                      0.111628
shelter                   0.088350
clothing                  0.015468
money                     0.023011
missing_people            0.011391
refugees                  0.033374
death                     0.045565
other_aid                 0.131362
infrastructure_related    0.064957
transport                 0.045832
buildings                 0.050861
electricity               0.020344
tools                     0.006058
hospitals                 0.010782
shops                     0.004572
aid_centers               0.011772
other_infrastructure      0.043851
weather_related     

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [79]:

    
def get_evaluation_metrics(actual, predicted, column_names, average='macro'):
    """
    Calculate evaluation metrics for a multi-label machine learning model.

    Args:
        actual (array-like): Ground truth labels (2D array or equivalent structure).
        predicted (array-like): Predicted labels from the model (2D array or equivalent structure).
        column_names (list of str): List of field names corresponding to each label.

    Returns:
        pd.DataFrame: A DataFrame containing Accuracy, Precision, Recall, and F1 Score for each label.
    """

    # Ensure inputs are NumPy arrays for consistent processing
    actual = np.asarray(actual)
    predicted = np.asarray(predicted)

    # Calculate metrics for each field and store in a list of dictionaries
    metrics = []
    for i, column in enumerate(column_names):
        metrics.append({
            "Field": column,
            "Accuracy": accuracy_score(actual[:, i], predicted[:, i]),
            "Precision": precision_score(actual[:, i], predicted[:, i], average=average),
            "Recall": recall_score(actual[:, i], predicted[:, i], average=average),
            "F1": f1_score(actual[:, i], predicted[:, i], average=average)
        })

    # Convert metrics into a DataFrame for better presentation
    metrics_df = pd.DataFrame(metrics).set_index("Field")
    return metrics_df

In [80]:
#Train
y_train_pred = pipeline.predict(X_train)
col_names = list(y.columns.values)

train_df = get_evaluation_metrics(y_train, y_train_pred, col_names, average='macro')
print(train_df)

#Test
y_test_pred = pipeline.predict(X_test)

test_df_first = get_evaluation_metrics(y_test, y_test_pred, col_names)
print( 'Test first model', test_df_first)



                        Accuracy        F1  Precision    Recall
Field                                                          
related                 0.984950  0.962562   0.991878  0.936868
request                 0.978854  0.959980   0.986896  0.937276
offer                   0.998666  0.895705   0.999332  0.827869
aid_related             0.971488  0.970313   0.975834  0.966139
medical_help            0.981077  0.928742   0.989435  0.883403
medical_products        0.986792  0.921833   0.990715  0.871292
search_and_rescue       0.993777  0.932011   0.996824  0.882775
security                0.995047  0.918164   0.997491  0.861210
military                0.991110  0.921458   0.995442  0.867675
child_alone             1.000000  1.000000   1.000000  1.000000
water                   0.983871  0.921726   0.991543  0.871066
food                    0.976124  0.932462   0.985602  0.892156
shelter                 0.978854  0.926446   0.988226  0.880798
clothing                0.996444  0.9360

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [81]:
#train_df.plot()
#plt.title("Plot for Training Data")
#test_df.plot()
#plt.title("Plot for Testing Data")
train_df.mean()


Accuracy     0.988106
F1           0.929196
Precision    0.992728
Recall       0.882896
dtype: float64

In [82]:
test_df_first.mean()

Accuracy     0.927410
F1           0.505910
Precision    0.544056
Recall       0.515145
dtype: float64


The significant difference between your training and test performance indicates overfitting and therefore using other methods to improve the model such as RandomForestClassifier.

### 6. Improve your model
Use grid search to find better parameters. 

In [86]:
from sklearn.ensemble import RandomForestClassifier


# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),          # Text to numerical conversion
    ('clf', RandomForestClassifier(n_estimators=100))  # Random Forest model
])

# Train the model
pipeline.fit(X_train, y_train)
print(X_train.isnull().sum())
print(X_train)
print(y_train)

0
14164    MERCY Malaysia's medical and humanitarian reli...
6797     Good morning,how are you? I would like to know...
2115     Saint Marc is full of victims, what is the gov...
16960    KABUL, 10 March (IRIN) - Following the harshes...
17964    The water container and faucets were set up at...
3501             NOTES: Is there a place that hire people?
8121     the state must subvention food for us to no bu...
251      my friend i have heard nothing again. i have a...
7363     I thank the number 4636 for these beautiful in...
22124    Prior to the floods, the majority of the popul...
24176    Many anticipated that it would generate large ...
15693    They also lack adequate medical equipment/supp...
23593    The policy, based on one developed for farmers...
8156               I would like to have helps with lizon. 
19598    Terrorist attacks or flare up of violence are ...
15955    The appearance of lava domes on April 28 was a...
12778    Finally coming out of my house!!! Checking ou

In [ ]:
# Fit the GridSearchCV
#try:
   # cv.fit(X_train, y_train)
    #print("Best Parameters:", cv.best_params_)
#except Exception as e:
    #print(f"CV failed with error: {e}")

In [ ]:
#np.random.seed(40)
#tuned_model = cv.fit(X_train, y_train)
#print(tuned_model.head())

In [ ]:
#tuned_model.cv_results_

In [ ]:
#np.max(tuned_model.cv_results_['mean_test_score'])

In [ ]:
#tuned_model.best_params_

Best parameters 
clf__estimator__min_samples_split=2, 
clf__estimator__n_estimators=50, 
tfidf__max_df=0.75, 
tfidf__ngram_range=(1, 1) 


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
tuned_pred_test = tuned_model.predict(X_test)
eval_metrics1 = get_evaluation_metrics(y_test, tuned_pred_test, col_names)

print(eval_metrics1)

In [ ]:
#eval_metrics0.describe()
#eval_metrics1.describe()

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.